In [1]:
AGENT_NAME = 'Models\Victim\SAC_citylearn_challenge_2022_phase_2_Building_6_default_rwd_MARLISA_hyperparams_500.zip'
DATASET_NAME = 'citylearn_challenge_2022_phase_2' #only action is electrical storage
SAVE_DIR = 'default SAC 500 norm space results' + '/'
ATK_NAME = 'untargeted_myPGD_03_mask_solar_and_time_scale_consumption_eps_clipped_adv_obs'
CONSUMPTION_IDX = 26
SOLAR_IDX = 24
CONSUMPTION_SPREAD = 0.016
SOLAR_SPREAD = 0.04
MIN_OBS = 0
MAX_OBS = 1

In [2]:
import torch
from torch import nn
from stable_baselines3 import SAC

from citylearn.data import DataSet

import pandas as pd
import numpy as np
import json

import KBMproject.utilities as utils

from tqdm import tqdm


In [3]:
schema = DataSet.get_schema(DATASET_NAME)

In [4]:
agent = SAC.load(path=f"{AGENT_NAME}")

In [5]:
from copy import deepcopy
policy_net = deepcopy(agent.actor.latent_pi) #copies shared net rather than referencing/changing the agent
policy_net.add_module('4', agent.actor.mu)

In [6]:
agent.actor.mu

Linear(in_features=256, out_features=1, bias=True)

In [7]:
env = utils.make_continuous_env(schema=schema,  
                        seed=42)
cols = env.observation_names

In [8]:
env.buildings[0].action_space

Box([-0.78125], [0.78125], (1,), float32)

In [9]:
env.action_space

C:\Users\kbrod\AppData\Roaming\Python\Python310\site-packages\gym\spaces\box.py:73: UserWarning: WARN: Box bound precision lowered by casting to float32
  logger.warn(


Box([0.], [1.], (1,), float32)

In [10]:
def broken_pgd_linf(model, X, y, loss_fn, epsilon:float=0.05, step:float=0.01, num_iter:int=100, 
             num_restarts:int=5, num_decay:int=0, decay_rate=1):
    """ Construct FGSM adversarial examples on the examples X with random restarts"""
    max_loss = torch.zeros([num_restarts, y.shape[0]]).to(y.device)
    max_delta = torch.zeros_like(X)

    assert 0 < decay_rate <= 1, 'decay rate must be between 0 and 1'

    if num_decay > 0: 
        decay_iters = num_iter//num_decay
    else: #no decay
        decay_iters = num_iter

    # Create a tensor to hold delta for all restarts at once
    delta = torch.rand(num_restarts, *X.shape, device=X.device, requires_grad=True)
    # Scale the random values to the range [-epsilon, epsilon]
    delta.data = delta.data * 2 * epsilon - epsilon

    for iter in range(num_iter):
        loss = loss_fn(reduction='none')(model(X + delta), y.unsqueeze(0).repeat(num_restarts, 1))
        loss.backward(torch.ones_like(loss))

        # Perform the update on delta (via the data attribute to skip the gradient tracking)
        delta.data = (delta + step*delta.grad.detach().sign()).clamp(-epsilon, epsilon)
        delta.grad.zero_()
        
        #find the best delta for all restarts
        is_max = loss.unsqueeze(-1).unsqueeze(-1) >= max_loss.unsqueeze(-1).unsqueeze(-1)
        max_delta = torch.where(is_max, delta.detach(), max_delta)
        max_loss = torch.where(is_max.squeeze(-1).unsqueeze(-1), loss, max_loss)

        if(iter%decay_iters == 0):
            step *= decay_rate
        
    return max_delta


In [11]:
def my_pgd_linf(model, X, y, loss_fn, epsilon:float=0.05, step:float=0.01, num_iter:int=100, 
                num_decay:int=0, decay_rate=1):
    """ Construct FGSM adversarial examples on the examples X with random restarts
    ref: https://adversarial-ml-tutorial.org/adversarial_examples/"""

    assert 0 < decay_rate <= 1, 'decay rate must be between 0 and 1'

    if num_decay > 0: 
        decay_iters = num_iter//num_decay
    else: #no decay
        decay_iters = num_iter

    delta = torch.zeros_like(X, requires_grad=True)
    for iter in range(num_iter):

        loss = loss_fn(reduction='none')(model(X + delta), y)
        loss.backward(torch.ones_like(loss))

        # Perform the update on delta (via the data attribute to skip the gradient tracking)
        delta.data = (delta + step*delta.grad.detach().sign()).clamp(-epsilon, epsilon)
        delta.grad.zero_()
        
        if(iter%decay_iters == 0):
            step *= decay_rate
        
    return delta


In [12]:
eps_list = np.ones(agent.observation_space.shape[0])*0.03
eps_list[:6] = 0.0 #masked
#these idx are improperly normalized, so eps bust be adjusted accordingly
eps_list[SOLAR_IDX] =0 #fully mask *= SOLAR_SPREAD
eps_list[CONSUMPTION_IDX] *= CONSUMPTION_SPREAD

In [13]:
kwargs = dict(
    model=policy_net,
    epsilon=torch.tensor(eps_list, device=agent.device, dtype=torch.float32),
    step=0.01,
    num_iter=100,
    #num_restarts=5,
    num_decay=4,
    decay_rate=0.5,
    loss_fn = nn.MSELoss
    #loss_fn = nn.HuberLoss,
    #loss_fn = nn.L1Loss #MAE loss
)

In [14]:
time_steps = None

obs_list = []
adv_obs_list = []
a_list = []
adv_a_list = []
mae = 0
n_features = agent.observation_space.shape[0]

observations = env.reset()
if time_steps is None:
    time_steps = env.time_steps - 1

pbar = tqdm(total=time_steps)
for step in tqdm(range(time_steps)):

    obs_list.append(observations)
    actions = agent.predict(observations, deterministic=True)
    a_list.append(actions[0])

    delta = my_pgd_linf(X=torch.from_numpy(observations).to(agent.device),
                                         y=torch.from_numpy(actions[0]).to(agent.device),
                                         **kwargs).cpu().detach().numpy()
    
    adv_obs = np.clip(observations + delta, MIN_OBS, MAX_OBS) #keep adv obs in obs space    adv_obs_list.append(adv_obs)
    adv_obs_list.append(adv_obs)
    
    a_adv, _ = agent.predict(adv_obs, deterministic=True)
    a_dist = abs(a_adv[0] - actions[0])[0]
    mae += a_dist

    adv_a_list.append(a_adv[0])
    observations, _, _, _ = env.step(a_adv)

    #update progress bar including MAE
    pbar.update(1)
    pbar.set_postfix({'MAE': mae/(step + 1)}, refresh=True)
    if env.done:
        break

pbar.close()
mae/=time_steps


100%|██████████| 8759/8759 [16:11<00:00,  9.02it/s, MAE=0.101]


In [15]:
kpi = utils.format_kpis(env)
display(kpi)

cost_function
annual_peak_average                      1.000000
carbon_emissions_total                   0.930463
cost_total                               0.883112
daily_one_minus_load_factor_average      1.117247
daily_peak_average                       0.982575
electricity_consumption_total            0.939536
monthly_one_minus_load_factor_average    1.001587
ramping_average                          1.131096
zero_net_energy                          1.057213
Name: District, dtype: float64

In [16]:
np.array(a_list).flatten().tolist()

[0.3048611283302307,
 0.4016590416431427,
 0.42396506667137146,
 0.4706825613975525,
 0.1041121780872345,
 0.4748331904411316,
 0.4969041347503662,
 0.5400773286819458,
 0.5702992677688599,
 0.559898316860199,
 0.5590198040008545,
 0.8697044849395752,
 0.9268227815628052,
 0.8067113161087036,
 0.7778151035308838,
 0.6190077662467957,
 0.9622252583503723,
 0.4786750376224518,
 0.3597843647003174,
 0.3390447199344635,
 0.3574584126472473,
 0.13304859399795532,
 0.15511047840118408,
 0.11888331174850464,
 0.07008013129234314,
 0.11494842171669006,
 0.32121679186820984,
 0.4291898012161255,
 0.4073580503463745,
 0.3564422130584717,
 0.5035580396652222,
 0.5309197306632996,
 0.5833287835121155,
 0.568851888179779,
 0.5918711423873901,
 0.5410409569740295,
 0.5522736310958862,
 0.5923742055892944,
 0.8204904794692993,
 0.9785014390945435,
 0.3802480697631836,
 0.5834833383560181,
 0.4478244185447693,
 0.4536959230899811,
 0.43602582812309265,
 0.4607819616794586,
 0.47841277718544006,
 0.463

In [17]:
kpi_savename = SAVE_DIR+'KPIs.csv'
try:
    df_kpis = pd.read_csv(kpi_savename, 
                          index_col=0)
    df_kpis[ATK_NAME] = kpi.values
    df_kpis.to_csv(kpi_savename)
    print('KPIs.csv updated')
except:
    kpi.name = ATK_NAME
    kpi.to_csv(kpi_savename)
    print('KPIs.csv created')

KPIs.csv updated


In [18]:
df_obs = pd.DataFrame(obs_list)
df_obs.columns = cols
df_obs['a'] = np.array(a_list).flatten().tolist()
df_obs.to_csv(SAVE_DIR+ATK_NAME+'_obs-a.csv')

In [22]:
adv_obs_list

[]

In [19]:
df_obs = pd.DataFrame(adv_obs_list)
df_obs.columns = cols
df_obs['a'] = np.array(adv_a_list).flatten().tolist()
df_obs.to_csv(SAVE_DIR+ATK_NAME+'_adv_obs-a.csv')

ValueError: Length mismatch: Expected axis has 0 elements, new values have 31 elements

In [23]:
asr_savename = SAVE_DIR+'MAEs.csv'
try:
    df_asrs = pd.read_csv(asr_savename)
    df_asrs[ATK_NAME] = mae
    df_asrs.to_csv(asr_savename)
    print(f'{asr_savename} updated')
except:
    asr = pd.Series([mae])
    asr.name = ATK_NAME
    asr.to_csv(asr_savename)
    print(f'{asr_savename} created')

default SAC 500 norm space results/MAEs.csv updated


In [25]:
kwargs_to_save = {k: v for k, v in kwargs.items() if k != 'model'} #don't save NN as json
kwargs_to_save['loss_fn'] = kwargs['loss_fn'].__name__ #replace function with a string
if not isinstance(kwargs_to_save['epsilon'], float):
    kwargs_to_save['epsilon'] = eps_list.tolist() #tensors aren't json compatible, use list
with open(SAVE_DIR+f'{ATK_NAME} parameters.json', 'w') as f:
    json.dump(kwargs_to_save, f)